In [2]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [ ]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

In [3]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

In [4]:
from langgraph.prebuilt import ToolNode
from spotify_tools import find_top_tracks, get_playlists, create_spotify_playlist, get_track_list_from_playlist, add_tracks_to_playlist, filter_artists, get_artists_from_playlist, find_similar_artists
from plan import validate_plan

tools = [get_playlists, create_spotify_playlist, add_tracks_to_playlist, filter_artists, validate_plan, get_artists_from_playlist, find_similar_artists, get_track_list_from_playlist, find_top_tracks]
# tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, search_tool]
tool_node = ToolNode(tools)

In [5]:
from langchain_core.messages import AIMessage
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "find_similar_artists",
            "args" : {"artists": ["spotify:artist:4uk9cWBqeiel7tO1w9MPe0", "spotify:artist:01aC2ikO4Xgb2LUpf9JfKp", "spotify:artist:4NpFxQe2UvRCAjto3JqlSl"]},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

In [7]:
response = tool_node.invoke({"messages": [message_with_single_tool_call]})
tool_message = response["messages"][0]
tool_message.pretty_print()

2024-12-02 23:49:07,426 - INFO - User authentication requires interaction with your web browser. Once you enter your credentials and give authorization, you will be redirected to a url.  Paste that url you were directed to to complete the authorization.
2024-12-02 23:49:07,506 - INFO - Opened https://accounts.spotify.com/authorize?client_id=73dde5f5fcc54ef999b16118c1703326&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A33333&scope=+playlist-modify-private++playlist-modify-public+user-library-modify in your browser
2024-12-02 23:49:10,731 - ERROR - HTTP Error for GET to https://api.spotify.com/v1/artists/4uk9cWBqeiel7tO1w9MPe0/related-artists with Params: {} returned 404 due to Not Found
2024-12-02 23:49:26,762 - ERROR - Unexpected error for artist spotify:artist:4uk9cWBqeiel7tO1w9MPe0: http status: 404, code:-1 - https://api.spotify.com/v1/artists/4uk9cWBqeiel7tO1w9MPe0/related-artists:
 Not Found, reason: None
2024-12-02 23:49:26,830 - ERROR - HTTP Error for GET to https://a

================================= Tool Message =================================
Name: find_similar_artists

{}


# Test find_top_tracks()

In [ ]:
from langchain_core.messages import AIMessage
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "find_top_tracks",
            "args" : {"artists": ["spotify:artist:6vYg01ZFt1nREsUDMDPUYX", "spotify:artist:4NpFxQe2UvRCAjto3JqlSl"]},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

In [ ]:
response = tool_node.invoke({"messages": [message_with_single_tool_call]})
tool_message = response["messages"][0]
# tool_message.pretty_print()
response

# Test find similar artists

In [ ]:
artist_uris = [
    "spotify:artist:01aC2ikO4Xgb2LUpf9JfKp", "spotify:artist:1iEaqWaYpKo9x0OrEq7Q7z",
    "spotify:artist:0RqtSIYZmd4fiBKVFqyIqD", "spotify:artist:7jdFEYD2LTYjfwxOdlVjmc",
    "spotify:artist:7Ln80lUS6He07XvHI8qqHH", "spotify:artist:4pejUc4iciQfgdX6OKulQn",
    "spotify:artist:0Nrwy16xCPXG8AwkMbcVvo", "spotify:artist:1xKrH6GSh9CJh8nYwbqW7B",
    "spotify:artist:6vYg01ZFt1nREsUDMDPUYX", "spotify:artist:0bZCak2tcRMY1dzEIuwF42",
    "spotify:artist:4NpFxQe2UvRCAjto3JqlSl", "spotify:artist:26opZSJcXshCmCwxgZQmBc",
    "spotify:artist:3xYXYzm9H3RzyQgBrYwIcx", "spotify:artist:13ZEDW6vyBF12HYcZRr4EV",
    "spotify:artist:4YLtscXsxbVgi031ovDDdh", "spotify:artist:5VPCIIfZPK8KPsgz4jmOEC",
    "spotify:artist:1l9I7G8J8AnMScWQwlNJ4M", "spotify:artist:12Chz98pHFMPJEknJQMWvI",
    "spotify:artist:5HsS48kuvghKcNpwOaAvB5", "spotify:artist:7mnBLXK823vNxN3UWB7Gfz",
    "spotify:artist:3mVWMgLc7bcyCBtL2ymZwK", "spotify:artist:2ziB7fzrXBoh1HUPS6sVFn",
    "spotify:artist:2gqMBdyddvN82dzZt4ZF14", "spotify:artist:2PqsalqLh5RT6FSy9DrGZh",
    "spotify:artist:26T3LtbuGT1Fu9m0eRq5X3", "spotify:artist:3jdirYL01245TvAH39GfM6",
    "spotify:artist:3TNt4aUIxgfy9aoaft5Jj2", "spotify:artist:2ueoLVCXQ948OfhVvAy3Nn",
    "spotify:artist:49qiE8dj4JuNdpYGRPdKbF", "spotify:artist:6hQS54VPpxunuwR0W7usuo",
    "spotify:artist:66YGDwn22fjphzqGCSIbbK", "spotify:artist:6lyMYewq2SuTFIXgiv7OxH",
    "spotify:artist:5tFRohaO5yEsuJxmMnlCO9", "spotify:artist:16GcWuvvybAoaHr0NqT8Eh",
    "spotify:artist:0qLNsNKm8bQcMoRFkR8Hmh", "spotify:artist:75dQReiBOHN37fQgWQrIAJ",
    "spotify:artist:3yEnArbNHyTCwMRvD9SBy4", "spotify:artist:4rfE3kN2zKNC9L9tt3iVOg",
    "spotify:artist:5P1oS9DUTPEqcrmXDmX4p8", "spotify:artist:3WyGjUsOZJRiT9re5pZYxL",
    "spotify:artist:7sMcWECQxLm7EoKdRlSCAn", "spotify:artist:4uk9cWBqeiel7tO1w9MPe0",
    "spotify:artist:4S47feOS2ATuhc7Ao5ilfG", "spotify:artist:14ZxDAK6ITtZZqPdiWrvSn",
    "spotify:artist:1moxjboGR7GNWYIMWsRjgG", "spotify:artist:7I3bkknknQkIiatWiupQgD",
    "spotify:artist:2qk9voo8llSGYcZ6xrBzKx", "spotify:artist:6GI52t8N5F02MxU0g5U69P",
    "spotify:artist:1fZpYWNWdL5Z3wrDtISFUH", "spotify:artist:6XyAI3jtlsss5mJjAAJXzE",
    "spotify:artist:5hPR4Atp3QY2ztiAcz1inl", "spotify:artist:5kuJibJcwOC53s3OkoGMRA",
    "spotify:artist:2x9SpqnPi8rlE9pjHBwmSC", "spotify:artist:7MhMgCo0Bl0Kukl93PZbYS",
    "spotify:artist:7J2lZBANizgPNfUzux31PV", "spotify:artist:6LufpoVlIYKQCu9Gjpk8B7",
    "spotify:artist:5rZUNweztKBI1Xy3XhYHoJ", "spotify:artist:246dkjvS1zLTtiykXe5h60",
    "spotify:artist:1zNqQNIdeOUZHb8zbZRFMX", "spotify:artist:22bE4uQ6baNwSHPVcDxLCe",
    "spotify:artist:1HY2Jd0NmPuamShAr6KMms", "spotify:artist:4xac3zhHlBm5QDxbZeqgeR",
    "spotify:artist:3lDpdwM8KILepMHqBWUhIA", "spotify:artist:3EaMbsBlExxNxLvTJcZvDq",
    "spotify:artist:0XHiH53dHrvbwfjYM7en7I", "spotify:artist:6cSxzHrQgGc4I4Ck5Gewej",
    "spotify:artist:69tiO1fG8VWduDl3ji2qhI", "spotify:artist:251UrhgNbMr15NLzQ2KyKq",
    "spotify:artist:1QAJqy2dA3ihHBFIHRphZj", "spotify:artist:2FXC3k01G6Gw61bmprjgqS",
    "spotify:artist:7jy3rLJdDQY21OgRLCZ9sD", "spotify:artist:6VDdCwrBM4qQaGxoAyxyJC",
    "spotify:artist:3kjuyTCjPG1WMFCiyc5IuB", "spotify:artist:49zZ2lRNpfwWfUnASUxCYW",
    "spotify:artist:4F84IBURUo98rz4r61KF70", "spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ",
    "spotify:artist:29mRqqZ15WaYjEsKNzcRkv", "spotify:artist:3hv9jJF3adDNsBSIQDqcjp",
    "spotify:artist:6Q192DXotxtaysaqNPy5yR", "spotify:artist:6YWdHD3R863Apw1hkx3BwC",
    "spotify:artist:66jsWaGhzSpHH1KRF34Oq3", "spotify:artist:6D6rjLdxyE5vwhMlkuQq0E",
    "spotify:artist:1CcPlAmcnJjC4FnaPVzv2v", "spotify:artist:6eU0jV2eEZ8XTM7EmlguK6",
    "spotify:artist:356c8AN5YWKvz86B4Sb1yf", "spotify:artist:33qOK5uJ8AR2xuQQAhHump",
    "spotify:artist:3JulrApLVT81sb2HkfwMks", "spotify:artist:3Z7thZHrtFvqp8OpPffPKp",
    "spotify:artist:090VebphoycdEyH165iMqc", "spotify:artist:1UfzhwcOR4yfX7yHTPfC9m",
    "spotify:artist:0MASTEXfUt3bpiyGOoEaur"
]


In [ ]:
from langgraph.prebuilt import ToolNode
from spotify_tools import find_similar_artists

tools = [find_similar_artists]
tool_node = ToolNode(tools)

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
import json

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=1.0)
# llm_with_tools = llm.bind_tools(tools, strict=True, parallel_tool_calls=False)
llm_with_tools = llm.bind_tools(tools, strict=True)
prompt = f"""
For each artist in the Spotify URI list below, find 3-4 artists of a similar music style

{json.dumps(artist_uris)}
"""
human_message = HumanMessage(prompt)

In [ ]:
ai_message = llm_with_tools.invoke([human_message])

In [ ]:
ai_message.pretty_print()

# Test Create Spotify

In [ ]:
from langchain_core.messages import AIMessage
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "create_spotify_playlist",
            "args" : {"name": "Test 3", "description": "A carefully curated collection of soulful rock and blues tracks, featuring rising stars and post-2010 successes, delivering a seamless and energetic listening experience"},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

In [ ]:
from langgraph.prebuilt import ToolNode
from spotify_tools import create_spotify_playlist

tools = [create_spotify_playlist]
tool_node = ToolNode(tools)

In [ ]:
response = tool_node.invoke({"messages": [message_with_single_tool_call]})
tool_message = response["messages"][0]
# tool_message.pretty_print()
response